<a href="https://colab.research.google.com/github/TezBytes/music-recommender/blob/feat%2Fdata-enrichment-track-genre/notebooks/02_data_enrichment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tezbytes/music-recommender

Cloning into 'music-recommender'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 18 (delta 1), reused 11 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 8.02 KiB | 8.02 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [8]:
%cd music-recommender


/content/music-recommender


In [9]:
!ls


notebooks  README.md  scripts


# Import music-recommender-data

In [26]:
import pandas as pd

df = pd.read_csv("hf://datasets/tezbytes/music-recommender-data/enriched_data.csv")

In [27]:
print(df.shape)

(89740, 22)


# Analyze track_genre column

In [28]:
df['track_genre'].value_counts().head(20)

,count
track_genre,
acoustic,1000
alt-rock,999
tango,999
ambient,999
afrobeat,999
cantopop,999
bluegrass,998
forro,998
study,998


# Clean multi-genre entries

In [29]:
def normalize_genre(genre):
  if pd.isna(genre):
    return "unknown"
  return genre.strip().lower()

# Genre Mapping

In [30]:
genre_map = {
    "accoustic": "pop",
    "alt-rock":  "rock",
    "tango": "world",
    "ambient": "electronic",
    "afrobeat": "world",
    "cantopop": "pop",
    "bluegrass": "folk",
    "forro": "world",
    "study": "ambient",
    "chicago-house": "electronic",
    "disney": "pop",
    "sleep": "ambient",
    "heavy-metal": "rock",
    "breakbeat": "electronic",
    "black-metal": "rock",
    "j-idol": "pop",
    "happy": "mood",
    "anime": "pop",
    "club": "electronic",
    "comedy": "spoken",
}

In [31]:
df["normalized_genre"] = df["track_genre"].apply(normalize_genre)
df["high_level_genre"] = df["normalized_genre"].apply(lambda g: genre_map.get(g, "other"))

In [34]:
df["normalized_genre"].value_counts().head()

,count
normalized_genre,
acoustic,1000
alt-rock,999
tango,999
ambient,999
afrobeat,999


In [35]:
df["high_level_genre"].value_counts().head()

,count
high_level_genre,
other,70799
electronic,3988
pop,3987
world,2996
rock,2992


In [36]:
df.to_csv("enriched_data.csv", index=False)

# Push to HF

In [40]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00


In [42]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `music-recommender-data-upload` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might ha

In [43]:
from datasets import Dataset
Dataset.from_pandas(df).push_to_hub("tezbytes/music-recommender-data")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/90 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tezbytes/music-recommender-data/commit/aa3fa3a7e02c7dce05c952fc33e133f3fee42394', commit_message='Upload dataset', commit_description='', oid='aa3fa3a7e02c7dce05c952fc33e133f3fee42394', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tezbytes/music-recommender-data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tezbytes/music-recommender-data'), pr_revision=None, pr_num=None)